# Analysis of Ubiquant data

This notebook reproduces this [Kaggle submission](https://www.kaggle.com/code/valleyzw/ubiquant-lgbm-baseline).

In [ ]:
import os
import gc
import sys
import joblib
import random
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime
from argparse import Namespace
from collections import defaultdict
from scipy.signal import find_peaks

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, GroupKFold, train_test_split, KFold

from utils import reduce_mem_usage
from lgbm_ops import rmse, feval_rmse, feval_pearsonr, weighted_average, run

import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

YELLOW = '#FFBC00'
GREEN = '#37795D'
PURPLE = '#5460C0'
BACKGROUND = '#F4EBE6'
colors = [GREEN, PURPLE]
custom_params = {
    'axes.spines.right': False, 'axes.spines.top': False,
    'axes.facecolor':BACKGROUND, 'figure.facecolor': BACKGROUND, 
    'figure.figsize':(8, 8)
}
sns_palette = sns.color_palette(colors, len(colors))
sns.set_theme(style='ticks', rc=custom_params)

def seed_everything(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
args = Namespace(
    debug=False,
    seed=21,
    folds=5,
    workers=4,
    min_time_id=None, 
    holdout=False,
    cv_method="group",
    num_bins=16,
    holdout_size=100,
    outlier_threshold=0.001,
    trading_days_per_year=250,   # chinese stock market trading days per year (roughly)
    add_investment_id_model=False,
    data_path=Path("."),
    just_eda=False,
)
seed_everything(args.seed)

if args.debug:
    setattr(args, 'min_time_id', 1150)

assert args.cv_method in {"kfold", "group", "stratified", "time", "group_time", "time_range"}, "unknown cv method"
assert args.data_path.exists(), "data_path not exists"

In [ ]:
train = pd.read_parquet(args.data_path.joinpath("train_low_mem.parquet"))
assert train.isnull().any().sum() == 0, "null exists."
assert train.row_id.str.extract(r"(?P<time_id>\d+)_(?P<investment_id>\d+)").astype(train.time_id.dtype).equals(train[["time_id", "investment_id"]]), "row_id!=time_id_investment_id"
assert train.time_id.is_monotonic_increasing, "time_id not monotonic increasing"

# Stock market calendar analysis: [discussion](https://www.kaggle.com/c/ubiquant-market-prediction/discussion/309720)
> The Chinese stock market turbulence began with the popping of the stock market bubble on 12 June 2015 and ended in early February 2016. - [wikipedia](https://en.wikipedia.org/wiki/2015%E2%80%932016_Chinese_stock_market_turbulence)

In [ ]:
calendar_df = pd.read_csv("./holidays_of_china_from_2014_to_2030.csv", parse_dates=["date"], date_parser=lambda x: datetime.strptime(x, '%Y-%m-%d'))
display(calendar_df.head())

# leave only national holidays
calendar_df = calendar_df.loc[(calendar_df.type.isin(["National holiday", "Common local holiday"]))]
display(calendar_df.head())

# fill with everyday from 2014 to 2022
calendar_df = (
    pd.DataFrame({"date": pd.date_range(start="2014-01-01", end="2022-01-01")}).merge(calendar_df, on="date", how="left")
    .assign(weekday=lambda x: x.date.dt.day_name(), year=lambda x: x.date.dt.year)
)
display(calendar_df.head())

# remove weekends and national holidays and align with time_id
calendar_df = (
    calendar_df.loc[(~calendar_df.weekday.isin(["Sunday", "Saturday"]))&(calendar_df.name.isna())]
    .reset_index(drop=True)
    .head(train.time_id.max()+1)
    .dropna(axis=1)
)
display(calendar_df.head())

# Target Analysis

In [ ]:
full_time_id_range = range(train.time_id.min(), train.time_id.max()+1)
f, (ax0, ax1, ax2) = plt.subplots(3, 1, gridspec_kw={'height_ratios': [1,1,6]}, figsize=(10,10), sharex=True, dpi=128)
_df = (
    train[['time_id', 'investment_id']]
    .groupby("time_id")
    .count()
    .reindex(full_time_id_range)
    .set_index(calendar_df.date)
)
peeks, _ = find_peaks(-_df.values.squeeze(), threshold=200)
_df.plot(ax=ax0, color=PURPLE)
ax0.set_xticks(ticks=_df.iloc[peeks].index.values, minor=True)
ax0.set_ylabel("count")
ax0.legend(loc='upper left')

(
    train[['time_id', 'target']]
    .groupby("time_id")
    .mean()
    .reindex(full_time_id_range)
    .set_index(calendar_df.date)
    .plot(ax=ax1, color=PURPLE)
)
ax1.axvspan(*mdates.date2num(_df.loc[(_df.index>"2015-06")&(_df.index<"2016-03")].index[[0,-1]]), fill=True, alpha=0.9, color=YELLOW)
ax1.set_ylabel("mean")
ax1.legend(loc='upper left')

_df = (
    train[['investment_id', 'time_id', "target"]]
    .pivot_table(index="time_id", columns="investment_id", values="target", aggfunc="count")
    .reindex(full_time_id_range)
    .set_index(calendar_df.date)
)
ax2.imshow(_df.T, cmap='winter', interpolation='nearest', aspect="auto", origin="lower", alpha=0.6, extent=[*mdates.date2num([calendar_df.date.min(), calendar_df.date.max()]), train.investment_id.min(), train.investment_id.max()])
ax2.set_xlabel("date")
ax2.set_ylabel("investment_id")
ax2.xaxis_date()
plt.tight_layout()
plt.show()

# Outliers

In [ ]:
df = train[["investment_id", "target"]].groupby("investment_id").target.mean()
upper_bound, lower_bound = df.quantile([1-args.outlier_threshold, args.outlier_threshold])
ax = df.plot(figsize=(16, 8), color=PURPLE)
ax.axhspan(lower_bound, upper_bound, fill=False, linestyle="--", color="k")
plt.show()

outlier_investments = df.loc[(df>upper_bound)|(df<lower_bound)|(df==0)].index
_=pd.pivot(
    train.loc[train.investment_id.isin(outlier_investments), ["investment_id", "time_id", "target"]],
    index='time_id', columns='investment_id', values='target'
).plot(figsize=(16,12), subplots=True, sharex=True)

#  Remove feature outliers: [notebook](https://www.kaggle.com/junjitakeshima/ubiquant-simple-lgbm-removing-outliers-en-jp)


In [ ]:
outlier_list = []
outlier_col = []

for col in (f"f_{i}" for i in range(300)):
    _mean, _std = train[col].mean(), train[col].std()
    
    temp_df = train.loc[(train[col] > _mean + _std * 70) | (train[col] < _mean - _std * 70)]
    temp2_df = train.loc[(train[col] > _mean + _std * 35) | (train[col] < _mean - _std * 35)]
    if len(temp_df) >0 : 
        outliers = temp_df.index.to_list()
        outlier_list.extend(outliers)
        outlier_col.append(col)
        print(col, len(temp_df))
    elif len(temp2_df)>0 and len(temp2_df) <6 :
        outliers = temp2_df.index.to_list()
        outlier_list.extend(outliers)
        outlier_col.append(col)
        print(col, len(temp2_df))

outlier_list = list(set(outlier_list))
train.drop(train.index[outlier_list], inplace = True)
print(len(outlier_col), len(outlier_list), train.shape)

# Missing time IDs

In [ ]:
_=pd.pivot(
    train.loc[train.investment_id.isin([1,17,19,3011,3151]), ["investment_id", "time_id", "target"]],
    index='time_id', columns='investment_id', values='target'
).plot(figsize=(16,12), subplots=True, sharex=True, color=GREEN)

# Interesting Time IDs

In [ ]:
_features = ["f_74", "f_153", "f_183", "f_145"]
df=train[["time_id", "target"]+_features].groupby("time_id").mean()
time_to_cheer_up, time_to_calm_down = df.target.idxmax(), df.target.idxmin()
ax, *_ = df.plot(figsize=(16,12), subplots=True, sharex=True, color=GREEN, alpha=.5)
ax.axhline(0, linestyle="--", color="k", linewidth=1)
ax.scatter(time_to_cheer_up, df.loc[time_to_cheer_up, "target"], marker="^", color=GREEN)
ax.scatter(time_to_calm_down, df.loc[time_to_calm_down, "target"], marker="v", color=GREEN)
plt.show()

In [ ]:
ax, *_ = train.loc[train.time_id==time_to_cheer_up, ["investment_id", "target"]+_features].plot(x="investment_id", figsize=(16,12), subplots=True, sharex=True, color=GREEN, alpha=.5)
ax.axhline(0, linestyle="--", color="k")
ax, *_ = train.loc[train.time_id==time_to_calm_down, ["investment_id", "target"]+_features].plot(x="investment_id", figsize=(16,12), subplots=True, sharex=True, color=GREEN, alpha=.5)
ax.axhline(0, linestyle="--", color="k")
plt.show()

In [ ]:
if args.min_time_id is not None:
    train = train.query("time_id>=@args.min_time_id").reset_index(drop=True)
    gc.collect()
    
train=train.loc[~train.investment_id.isin(outlier_investments)].reset_index(drop=True)
train.shape

In [ ]:
_=train[["time_id", "target"]+_features].groupby("time_id").mean().plot(figsize=(16,12), subplots=True, sharex=True, color=GREEN, alpha=.5)

In [ ]:
time_id_df = (
    train[["investment_id", "time_id"]]
    .groupby("investment_id")
    .agg(["min", "max", "count", np.ptp])
    .assign(
        time_span=lambda x: x.time_id.ptp,
        time_count=lambda x: x.time_id["count"]
    )
    .drop(columns="ptp", level=1)
    .reset_index()
)
time_id_df.head(6)

In [ ]:
train = train.merge(time_id_df.drop(columns="time_id", level=0).droplevel(level=1, axis=1), on="investment_id", how='left')
train[["time_span", "time_count"]].hist(bins=args.num_bins, figsize=(16,12), sharex=True, layout=(2,1), color=GREEN)
max_time_span=time_id_df.time_id["max"].max()
outlier_investments = time_id_df.loc[time_id_df.time_id["count"]<32, "investment_id"].to_list()
del time_id_df
gc.collect()

In [ ]:
if args.holdout:
    _target = pd.cut(train.time_span, args.num_bins, labels=False)
    _train, _valid = train_test_split(_target, stratify=_target, random_state=args.seed)
    print(f"train length: {len(_train)}", f"holdout length: {len(_valid)}")
    valid = train.iloc[_valid.index].sort_values(by=["time_id", "investment_id"]).reset_index(drop=True)
    train = train.iloc[_train.index].sort_values(by=["time_id", "investment_id"]).reset_index(drop=True)
    train.time_span.hist(bins=args.num_bins, figsize=(16,8), alpha=0.8)
    valid.time_span.hist(bins=args.num_bins, figsize=(16,8), alpha=0.8)
    valid.drop(columns="time_span").to_parquet("valid.parquet")
    del valid, _train, _valid, _target
    gc.collect()
assert train.time_id.is_monotonic_increasing, "time_id not monotonic increasing"

In [ ]:
if args.cv_method=="stratified":
    train["fold"] = -1
    _target = pd.cut(train.time_span, args.num_bins, labels=False)
    skf = StratifiedKFold(n_splits=args.folds)
    for fold, (train_index, valid_index) in enumerate(skf.split(_target, _target)):
        train.loc[valid_index, 'fold'] = fold

    fig, axs = plt.subplots(nrows=args.folds, ncols=1, sharex=True, figsize=(16,8), tight_layout=True)
    for ax, (fold, df) in zip(axs, train[["fold", "time_span"]].groupby("fold")):
        ax.hist(df.time_span, bins=args.num_bins)
        ax.text(0, 40000, f"fold: {fold}, count: {len(df)}", fontsize=16)
    plt.show()
    del _target, train_index, valid_index
    _=gc.collect()
if args.just_eda:
    sys.exit(0)

In [ ]:
cat_features = []
num_features = list(train.filter(like="f_").columns)
features = num_features + cat_features

combination_features = ["f_231-f_250", "f_118-f_280", "f_155-f_297", "f_25-f_237", "f_179-f_265", "f_119-f_270", "f_71-f_197", "f_21-f_65"]
for f in combination_features:
    f1, f2 = f.split("-")
    train[f] = train[f1] + train[f2]
features += combination_features

to_drop = ["f_148", "f_72", "f_49", "f_205", "f_228", "f_97", "f_262", "f_258"]
features = list(sorted(set(features).difference(set(to_drop))))

train = reduce_mem_usage(train.drop(columns="time_span"))
train[["investment_id", "time_id"]] = train[["investment_id", "time_id"]].astype(np.uint16)
train=train.drop(columns=["row_id"]+to_drop)

if args.cv_method=="stratified":
    train["fold"] = train["fold"].astype(np.uint8)
gc.collect()
#features += ["time_id"] # https://www.kaggle.com/c/ubiquant-market-prediction/discussion/302429
features_backup = features.copy()
len(features)

In [ ]:
if args.cv_method=="time_range":
    train["time_range"] = pd.cut(train.time_id, bins=int(np.ceil(max_time_span/args.trading_days_per_year)))
    _ = train.time_range.value_counts(sort=False).plot(kind="barh", figsize=(16,8))

In [ ]:
from lgbm_ops import rmse, feval_rmse, feval_pearsonr, weighted_average, run

In [ ]:
investment_ids = train.investment_id.unique()
info = "without_investment_id"
features_importance = run(info=info, args=args, train=train, features=features, cat_features=cat_features)
df = train[["target", "preds", "time_id"]].query("preds!=-1000")
score = df.groupby("time_id").apply(lambda x: pearsonr(x.target, x.preds)[0]).mean()
print(f"lgbm {info} {args.cv_method} {args.folds} folds mean rmse: {rmse(df.target, df.preds):.4f}, mean pearsonr: {pearsonr(df.target, df.preds)[0]:.4f}, mean pearsonr by time_id: {score:.4f}")

folds_mean_importance = (
    features_importance.groupby("feature", as_index=False)
    .importance.mean()
    .sort_values(by="importance", ascending=False)
)
features_importance.to_csv(f"features_importance_{info}.csv", index=False)
folds_mean_importance.to_csv(f"folds_mean_feature_importance_{info}.csv", index=False)

plt.figure(figsize=(16, 10))
plt.subplot(1,2,1)
sns.barplot(x="importance", y="feature", data=folds_mean_importance.head(50))
plt.title(f'Head LightGBM Features {info} (avg over {args.folds} folds)')
plt.subplot(1,2,2)
sns.barplot(x="importance", y="feature", data=folds_mean_importance.tail(50))
plt.title(f'Tail LightGBM Features {info} (avg over {args.folds} folds)')
plt.tight_layout()
plt.show()

del df
gc.collect()

In [ ]:
if args.add_investment_id_model:
    cat_features = ["investment_id"]
    features += cat_features
    info = "with_investment_id"
    features_importance = run(info=info, args=args, train=train, features=features, cat_features=cat_features)
    df = train[["target", "preds", "time_id"]].query("preds!=-1000")
    score = df.groupby("time_id").apply(lambda x: pearsonr(x.target, x.preds)[0]).mean()
    print(f"lgbm {info} {args.cv_method} {args.folds} folds mean rmse: {rmse(df.target, df.preds):.4f}, mean pearsonr: {pearsonr(df.target, df.preds)[0]:.4f}, mean pearsonr by time_id: {score:.4f}")

    folds_mean_importance = (
        features_importance.groupby("feature", as_index=False)
        .importance.mean()
        .sort_values(by="importance", ascending=False)
    )
    features_importance.to_csv(f"features_importance_{info}.csv", index=False)
    folds_mean_importance.to_csv(f"folds_mean_feature_importance_{info}.csv", index=False)

    plt.figure(figsize=(16, 10))
    plt.subplot(1,2,1)
    sns.barplot(x="importance", y="feature", data=folds_mean_importance.head(50))
    plt.title(f'Head LightGBM Features {info} (avg over {args.folds} folds)')
    plt.subplot(1,2,2)
    sns.barplot(x="importance", y="feature", data=folds_mean_importance.tail(50))
    plt.title(f'Tail LightGBM Features {info} (avg over {args.folds} folds)')
    plt.tight_layout()
    plt.show()
    del df

del train
gc.collect()

# Resources 
- [Ubiquant LGBM Baseline](https://www.kaggle.com/code/valleyzw/ubiquant-lgbm-baseline)